<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/playground-009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 必要ライブラリ・コマンドの導入
!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
w = !apt install tree
print(w[-2])

Setting up tree (2.0.2-1) ...


In [1]:
import torch
# レイヤー関数定義

# 最初の線形関数
# 784 入力数
# 128 出力数
l1 = torch.nn.Linear(784, 128)

# ２番目の線形関数
# 128 入力数
#  10 出力数
l2 = torch.nn.Linear(128, 10)

# 活性化関数
relu = torch.nn.ReLU(inplace=True)




In [5]:
# 入力テンソルから出力テンソルを計算

# ダミー入力データを作成
inputs = torch.randn(100, 784)

# 中間テンソル1の計算
m1 = l1(inputs)

# 中間テンソル2の計算
m2 = relu(m1)

# 出力テンソルの計算
outputs = l2(m2)

# 入力テンソルと出力テンソルのshape確認
print(f'入力テンソル:{inputs.shape}')
print(f'出力テンソル:{outputs.shape}')

# # 入力テンソルと出力テンソル確認
# print(f'入力テンソル:{inputs}')
# print(f'出力テンソル:{outputs}')



入力テンソル:torch.Size([100, 784])
出力テンソル:torch.Size([100, 10])
入力テンソル:tensor([[-0.8874,  0.4105, -2.0451,  ...,  0.9141, -1.4086,  0.3307],
        [ 0.0286, -1.1390, -0.1955,  ..., -1.1115, -1.1897,  0.2905],
        [-0.6946, -0.1835,  0.4338,  ...,  0.7818, -0.1373,  0.0589],
        ...,
        [-1.5647,  0.1948,  0.8232,  ...,  0.7477,  1.5727,  0.2438],
        [ 0.9025, -0.3414,  0.2369,  ...,  0.2950,  1.6278, -0.9148],
        [ 0.7599,  1.4982, -0.5615,  ...,  0.3669, -0.6021, -0.5901]])
出力テンソル:tensor([[-0.0131, -0.2462,  0.0672, -0.0140, -0.3052, -0.2824, -0.4100, -0.0657,
          0.0593,  0.0634],
        [ 0.1507, -0.0260, -0.2242, -0.0044,  0.0422,  0.1124,  0.0430, -0.2696,
          0.2553, -0.0030],
        [ 0.2875, -0.3479,  0.3824, -0.1431,  0.1858, -0.0520,  0.1494, -0.5671,
         -0.1303, -0.0644],
        [ 0.0273,  0.0338,  0.2962, -0.2724, -0.0978, -0.5147, -0.0206, -0.2092,
          0.2205, -0.3645],
        [-0.1404, -0.0675, -0.1126,  0.2171,  0.1880, -0.